In [1]:
import requests
import csv
import time
from datetime import datetime

Trying to get users of desired choice at same code but taking too much time

In [7]:
BASE_URL = 'https://codeforces.com/api/'

# Define your Codeforces handle (replace with your own handle)
your_handle = 'agrim07'

# Define the initial and target rating
initial_rating = 1200
target_rating = 1400

# Define the minimum and maximum number of days
min_days = 20
max_days = 40

# Define the minimum number of contests
min_contests = 3

# Define the batch size for retrieving data
batch_size = 50

# Define the output directory
output_directory = 'codeforces_users/'

# Create the output directory if it doesn't exist
import os
os.makedirs(output_directory, exist_ok=True)

current_file_count = 1

def get_user_info(handle):
    response = requests.get(BASE_URL + 'user.info', params={'handles': handle})
    if response.status_code != 200:
        return None
    data = response.json()
    if data['status'] == 'OK':
        return data['result'][0]
    return None

def get_users_in_range(initial_rating, target_rating, min_days, max_days, min_contests, count, offset):
    response = requests.get(BASE_URL + 'user.ratedList', params={'activeOnly': True})
    if response.status_code != 200:
        return None
    data = response.json()

    if data['status'] == 'OK':
        users = []

        for user in data['result']:
            if user['rating'] >= initial_rating and user['rating'] <= target_rating:
                handle = user['handle']
                user_info = get_user_info(handle)

                if user_info is not None:
                    registration_time = user_info['registrationTimeSeconds']
                    last_online_time = user_info['lastOnlineTimeSeconds']
                    now = time.mktime(datetime.now().timetuple())

                    # Calculate the number of days since registration
                    days_since_registration = (now - registration_time) / 86400

                    # Calculate the number of days since the last contest
                    days_since_last_contest = (now - last_online_time) / 86400

                    if min_days <= days_since_registration <= max_days and days_since_last_contest <= max_days:
                        response = requests.get(BASE_URL + 'user.rating', params={'handle': handle})
                        if response.status_code != 200:
                            continue
                        try:
                            rating_data = response.json()
                        except ValueError as e:
                            print(f"Error parsing JSON for user {handle}: {e}")
                            continue
                        if len(rating_data['result']) >= min_contests:
                            users.append({
                                'handle': handle,
                                'rating': user['rating'],
                                'registration_time': datetime.fromtimestamp(registration_time),
                                'last_online_time': datetime.fromtimestamp(last_online_time)
                            })

        return users

def save_users_to_csv(users):
    global current_file_count
    output_file = f'{output_directory}codeforces_users_{current_file_count}.csv'
    with open(output_file, mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(['Handle', 'Rating', 'Registration Time', 'Last Online Time'])
        for user in users:
            writer.writerow([user['handle'], user['rating'], user['registration_time'], user['last_online_time']])
    current_file_count += 1

offset = 1
user_count = 0
while True:
    users_batch = get_users_in_range(initial_rating, target_rating, min_days, max_days, min_contests, batch_size, offset)
    if users_batch is None:
        break

    if not users_batch:
        break

    save_users_to_csv(users_batch)
    user_count += len(users_batch)

    if user_count >= 100:
        user_count = 0

print("Data retrieval complete. Users saved to separate CSV files in", output_directory)

KeyboardInterrupt: 

Tried to get users above 1400 but did not make it batch wise

In [12]:
BASE_URL = 'https://codeforces.com/api/'

# Define the batch size for retrieving user data
batch_size = 1000

# Define the output CSV file
output_file = 'codeforces_usernames.csv'

def get_all_usernames():
    offset = 1
    usernames = []

    while True:
        response = requests.get(BASE_URL + 'user.ratedList', params={'activeOnly': True, 'count': batch_size, 'offset': offset})
        data = response.json()

        if data['status'] == 'OK':
            for user in data['result']:
                usernames.append(user['handle'])
            
            # Check if there are more users to fetch
            if len(data['result']) < batch_size:
                break
            else:
                offset += batch_size
        else:
            print("Error fetching data from Codeforces API. Exiting.")
            break

    return usernames

def save_usernames_to_csv(usernames):
    with open(output_file, mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(['Username'])
        for username in usernames:
            writer.writerow([username])

try:
    usernames = get_all_usernames()
    save_usernames_to_csv(usernames)
    print("Usernames of all Codeforces users saved to", output_file)
except requests.exceptions.RequestException as e:
    print(f"An error occurred while making a request to the Codeforces API: {e}")
except ValueError as e:
    print(f"An error occurred while processing JSON response: {e}")





KeyboardInterrupt: 

Same but did gave same csv file at every time

In [15]:
import requests
import csv
import time

# Define the Codeforces API base URL
BASE_URL = 'https://codeforces.com/api/'

# Define the batch size for retrieving user data
batch_size = 20

# Define the output directory
output_directory = 'codeforces_users_batches/'

# Create the output directory if it doesn't exist
import os
os.makedirs(output_directory, exist_ok=True)

def get_users_in_batches():
    offset = 1
    batch_number = 1

    while True:
        response = requests.get(BASE_URL + 'user.ratedList', params={'activeOnly': True, 'count': batch_size, 'offset': offset})
        data = response.json()

        if data['status'] == 'OK':
            users = []

            for user in data['result']:
                if user['rating'] > 1400:
                    users.append(user['handle'])
            
            if users:
                save_users_to_csv(users, batch_number)
                batch_number += 1

            # Check if there are more users to fetch
            if len(data['result']) < batch_size:
                break
            else:
                offset += batch_size
        else:
            print("Error fetching data from Codeforces API. Exiting.")
            break

def save_users_to_csv(users, batch_number):
    output_file = f'{output_directory}codeforces_users_batch_{batch_number}.csv'
    with open(output_file, mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(['Username'])
        for username in users:
            writer.writerow([username])

try:
    get_users_in_batches()
    print("Codeforces users with a rating above 1400 saved to separate CSV files in", output_directory)
except requests.exceptions.RequestException as e:
    print(f"An error occurred while making a request to the Codeforces API: {e}")
except ValueError as e:
    print(f"An error occurred while processing JSON response: {e}")


KeyboardInterrupt: 

Same user to be ommitted in csv files
Let it run for  more than 45 mins

In [16]:
import requests
import csv
import time

# Define the Codeforces API base URL
BASE_URL = 'https://codeforces.com/api/'

# Define the batch size for retrieving user data
batch_size = 20

# Define the output directory
output_directory = 'codeforces_users_batches/'

# Create the output directory if it doesn't exist
import os
os.makedirs(output_directory, exist_ok=True)

# Keep track of processed usernames to avoid duplicates
processed_usernames = set()

def get_users_in_batches():
    offset = 1
    batch_number = 1

    while True:
        response = requests.get(BASE_URL + 'user.ratedList', params={'activeOnly': True, 'count': batch_size, 'offset': offset})
        data = response.json()

        if data['status'] == 'OK':
            users = []

            for user in data['result']:
                if user['rating'] > 1400 and user['handle'] not in processed_usernames:
                    users.append(user['handle'])
                    processed_usernames.add(user['handle'])
            
            if users:
                save_users_to_csv(users, batch_number)
                batch_number += 1

            # Check if there are more users to fetch
            if len(data['result']) < batch_size:
                break
            else:
                offset += batch_size
        else:
            print("Error fetching data from Codeforces API. Exiting.")
            break

def save_users_to_csv(users, batch_number):
    output_file = f'{output_directory}codeforces_users_batch_{batch_number}.csv'
    with open(output_file, mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(['Username'])
        for username in users:
            writer.writerow([username])

try:
    get_users_in_batches()
    print("Codeforces users with a rating above 1400 saved to separate CSV files in", output_directory)
except requests.exceptions.RequestException as e:
    print(f"An error occurred while making a request to the Codeforces API: {e}")
except ValueError as e:
    print(f"An error occurred while processing JSON response: {e}")


KeyboardInterrupt: 

From the above 1400,let it see who have crossed 1200 to 1400 in 2 to 5 contests

In [22]:
import requests
import csv

BASE_URL = 'https://codeforces.com/api/'

input_csv_file = 'codeforces_users_batches/codeforces_users_batch_1.csv'
output_csv_file = 'filtered_users.csv'

min_rating = 1200
max_rating = 1400
min_contests = 2
max_contests = 5

def meets_criteria(user_info):
    if 'rating' in user_info and 'contests' in user_info:
        rating = user_info['rating']
        contest_count = len(user_info['contests'])

        return min_rating <= rating <= max_rating and min_contests <= contest_count <= max_contests
    return False

def get_user_info(username):
    response = requests.get(BASE_URL + 'user.info', params={'handles': username})
    if response.status_code == 200:
        data = response.json()
        if data['status'] == 'OK':
            return data['result'][0]
    return None

usernames = []
with open(input_csv_file, 'r') as csv_file:
    csv_reader = csv.reader(csv_file)
    for row in csv_reader:
        usernames.append(row[0])
filtered_users = []

for i, username in enumerate(usernames, start=1):
    user_info = get_user_info(username)
    if user_info and meets_criteria(user_info):
        filtered_users.append(user_info['handle'])
        print(f"Processed user {i}: {username}")

with open(output_csv_file, 'w', newline='') as csv_file:
    csv_writer = csv.writer(csv_file)
    csv_writer.writerow(['Username'])

    for username in filtered_users:
        csv_writer.writerow([username])

print(f"Filtered users saved to {output_csv_file}")



KeyboardInterrupt: 